# Objet : Test opendata des données IRVE

## Objectif

- valider sur des cas réels l'outil de traitement des "listes indexées"
- identifier les apports que pourraient avoir ce type d'outil

## Résultats
- l'outil fonctionne correctement (pas d'erreur identifiées, les temps de réponse restent néanmoins à améliorer).
- l'analyse met en évidence la structuration des données et le niveau de qualité des données 
- les données anormales sont idfentifiées 
- le gain en taille de fichier varie de 50% (format texte non optimisé) à 80% (taille divisée par 5 !) dans un format optimisé et binaire. Ce niveau d'optimisation est tout à fait notable.
- l'analyse permet de (re)trouver la logique de dépendance entre les colonnes qui minimise les incohérences
- elle permet également de vérifier que les données respectent bien une structure imposée

## Usages possibles 
- les indicateurs utilisés permettent de qualifier le fichier csv et sont déployables simplement:
    - niveau de duplication des données
    - lien de dépendance entre les colonnes (permet un codage réduit des données).
- l'usage de format de données moins gourmand (sans dégradation des données) peut également être intéressant à déployer
- les schémas de données pourraient intégrer cette dépendance entre colonnes (ex. s'il y a une colonne "mois" et une colonne "trimestre", on peut indiquer que la colonne "trimestre" est "dérivée" de la colonne "mois", ou bien si on a une colonne "nom prénom" et une colonne "n° sécurité sociale", on peut indiquer que les deux colonnes sont "couplées". 
- la qualité des données peut faire l'objet d'un indicateur qui mesure l'écart (existant / attendu) des relations entre colonnes
- l'identification des enregistrements ne respectant pas une structure imposée permet de les éliminer ou de les corriger

données utilisées : https://static.data.gouv.fr/resources/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/20220629-080611/consolidation-etalab-schema-irve-v-2.0.2-20220628.csv

------
## Initialisation
- lecture du fichier issu de l'api (quelques lignes incohérentes au niveau csv ont été éliminées au préalable)

In [1]:
from pprint import pprint
from collections import Counter
from time import time
import csv, os, requests
from copy import copy
from observation import Ilist, ES, util


chemin = 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/master/python/Validation/irve/'
data_csv = 'consolidation-etalab-schema-irve-v-2.0.2-20220606-propre2.csv'
url = chemin + data_csv
open(data_csv, 'wb').write(requests.get(url, allow_redirects=True).content)


print('file size : ', os.stat(data_csv).st_size)

file size :  7456077


In [2]:
with open(data_csv, newline='', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=';')
    names = next(reader)
    data = []
    for row in reader: data.append(row)
data2 = util.list(list(zip(*data)))

----
## initialisation de l'objet Ilist
- l'initialisation pourrait être automatisée à partir du fichier csv
- identification de 64 775 valeurs différentes sur un total de 11 163 x 49 valeurs ("taux d'unicité" de 12%)
- la taille minimale serait de 1,4 Mo (données csv "quotées") pour un maximum de 9,6 Mo (données csv "quotées")

In [3]:
t0=time()
idxs = Ilist.ext(data2, names)
print('idxs (len, lenlidx, sumcodec) : ', len(idxs), len(idxs.idxlen), sum(idxs.idxlen), time()-t0)
t0=time()
fullsize = len(idxs.to_obj(encoded=True, modecodec='full'))
print('\nfullsize', fullsize, time()-t0)
t0=time()
minsize = len(idxs.to_obj(encoded=True, modecodec='nokeys'))
print('\nminsize', minsize, time()-t0)

idxs (len, lenlidx, sumcodec) :  11163 49 64775 21.054966688156128

fullsize 9682078 2.5403573513031006

minsize 1390520 0.29297590255737305


----
## format non optimisé
- le "taux d'unicité"(unicity level) reste à 12% (pas de modification des index)
- le "taux de codage"(object lightness) est de 30% (remplacement des données dupliquées par un entier)
- le gain de taille de fichier par rapport à un fichier "quoté" est de 62%

In [4]:
t0=time()
defaultsize = len(idxs.to_obj(encoded=True, modecodec='default'))
print('defaultsize', defaultsize, time()-t0, '\n')
print('indicator default : ')
pprint(idxs.indicator(fullsize, defaultsize))
print('\n', idxs==idxs.from_obj(idxs.to_obj(encoded=True, modecodec='default')))

defaultsize 3701134 0.43479466438293457 

indicator default : 
{'gain': 0.618,
 'maxgain': 0.884,
 'mean coding size': 5.223,
 'mean size': 17.347,
 'object lightness': 0.301,
 'optimize level': 0.382,
 'total values': 558150,
 'unicity level': 0.116,
 'unique values': 64824}

 True


- l'analyse de la structure montre qu'aucun champ est de type 'primary' et que les données sont principalement du type "secondary". On est donc sur une structure non matricielle.

In [5]:
print(Counter(idxs.category.values()))

Counter({'secondary': 48, 'coupled': 1})


- l'examen de "l'arbre de dérivation" indique un faible niveau d'interdépendance entre champs (seuls quelques-uns ne sont pas liés à la racine). Par exemple les index longitude(43) et latitude(44) sont bien dérivés de l'index coordonneesXY(13)


In [6]:
print(idxs.tree())

-1: root-derived (11163)
   0 : nom_amenageur (481)
   1 : siren_amenageur (670)
   2 : contact_amenageur (388)
   3 : nom_operateur (152)
   4 : contact_operateur (159)
   5 : telephone_operateur (215)
   6 : nom_enseigne (764)
   7 : id_station_itinerance (7060)
   8 : id_station_local (3838)
   9 : nom_station (4503)
   10: implantation_station (5)
   11: adresse_station (4392)
      28: condition_acces (3)
   12: code_insee_commune (2593)
   13: coordonneesXY (4459)
      33: station_deux_roues (7)
      43: consolidated_longitude (4448)
      44: consolidated_latitude (4429)
   14: nbre_pdc (32)
   15: id_pdc_itinerance (10322)
   16: id_pdc_local (7095)
   17: puissance_nominale (61)
   18: prise_type_ef (9)
   19: prise_type_2 (8)
   20: prise_type_combo_ccs (8)
   21: prise_type_chademo (7)
   22: prise_type_autre (8)
   23: gratuit (9)
   24: paiement_acte (9)
   25: paiement_cb (8)
   26: paiement_autre (9)
   27: tarification (128)
   29: reservation (8)
   30: horaires (140

- le taux de couplage (cf ci-dessous) pour chacun des index est très proche de 0, ce qui signifie que les champs devraient être de type "derived" (lien de dépendance par exemple comme entre les trimestres et les mois) ou "coupled" (lien biunivoque). 
- ceci indique la présence probable d'erreurs de documentation du tableau

In [7]:
print(idxs.tree(mode='diff'))

-1: root-diff (11163)
   15: id_pdc_itinerance (1.00e+00 - 10322)
      6 : nom_enseigne (2.89e-05 - 764)
      7 : id_station_itinerance (5.76e-07 - 7060)
         14: nbre_pdc (8.22e-05 - 32)
         48: consolidated_is_code_insee_verified (1.42e-04 - 3)
      8 : id_station_local (7.04e-06 - 3838)
      9 : nom_station (5.94e-06 - 4503)
         0 : nom_amenageur (6.01e-06 - 481)
         1 : siren_amenageur (3.32e-06 - 670)
         2 : contact_amenageur (2.87e-06 - 388)
         3 : nom_operateur (1.91e-05 - 152)
         4 : contact_operateur (1.41e-05 - 159)
         12: code_insee_commune (7.71e-07 - 2593)
         13: coordonneesXY (2.19e-06 - 4459)
            5 : telephone_operateur (4.72e-05 - 215)
            11: adresse_station (2.09e-06 - 4392)
               28: condition_acces (0.00e+00 - 3)
               30: horaires (3.93e-05 - 140)
            33: station_deux_roues (0.00e+00 - 7)
            35: num_pdl (9.66e-06 - 1371)
            43: consolidated_longitude (0.

----
## couplage
Le couplage consiste à tranformer les index qui sont "presque couplés" en index couplés (tous les index sont bien maintenant de type "derived").    
    
Le choix des index à coupler s'effectue en fonction d'une mesure de "distance" entre deux index. Les index avec une distance inférieure à un seuil choisi sont alors couplés.


In [8]:
coup = copy(idxs)
coup.reindex()

t0=time()
coup.coupling(param='distance', level=1000)
print('coupling', time()-t0)

coupling 31.759572982788086


## Format optimisé
Le couplage effectué permet d'optimiser de nouveau le volume des données :    
- le "taux d'unicité" se dégrade légèrement (passage de 11,6% à 12,1%) par l'ajout d'index supplémentaires
- le "taux de codage" par contre passe de 30% à 16% de par l'optimisation 
- le gain de taille de fichier par rapport à un fichier "quoté" est maintenant de 74%
- l'utilisation d'un format binaire (codage CBOR pour Concise Binary Object Representation RFC 8949) permet d'améliorer encore le gain de taille de fichier (82%)    
    
En synthèse, on passe donc d'un volume de données de 9,7 Mo (données csv quotées) à un volume de 3,7 Mo (données textuelles formatées) puis 2,4 Mo (données textuelles optimisées) et enfin à 1,8 Mo (données binaires optimisées).

In [9]:
t0=time()
optimize = coup.to_obj(encoded=True)
print('optimizesize :\n', len(optimize), time()-t0, '\n')
print('indicator optimize :\n', coup.indicator(fullsize, len(optimize)), '\n')

t0=time()
cbor = coup.to_obj(encoded=True, encode_format='cbor')
cborsize = len(cbor)
print('cborsize :\n', cborsize, time()-t0, '\n')
print('indicator cbor : \n', coup.indicator(fullsize, cborsize))

optimizesize :
 2431290 8.861904382705688 

indicator optimize :
 {'total values': 558150, 'mean size': 17.347, 'unique values': 78004, 'mean coding size': 2.246, 'unicity level': 0.14, 'optimize level': 0.251, 'object lightness': 0.129, 'maxgain': 0.86, 'gain': 0.749} 

cborsize :
 1798910 0.6935088634490967 

indicator cbor : 
 {'total values': 558150, 'mean size': 17.347, 'unique values': 78004, 'mean coding size': 0.928, 'unicity level': 0.14, 'optimize level': 0.186, 'object lightness': 0.054, 'maxgain': 0.86, 'gain': 0.814}


----
## Intégrité
- la transformation inverse des données binaires permet de vérifier que les données optimisées sont bien identiques aux données initiales (pas de dégradation, reversibilité).

In [10]:
t0=time()
idxs2 = Ilist.from_obj(cbor)
print('fromcbor', len(idxs2), time()-t0)

t0=time()
verif = idxs2 == idxs
print('controle égalité :', verif, time()-t0)

fromcbor 11163 9.942213535308838
controle égalité : True 0.3523378372192383


----
## Exemple de données anormales
L'analyse précédente met en évidence des incohérences entre données illustrées par quelques exemples ci-dessous :     
- l'index "coordonneesXY"(13) est lié à l'index "nom_station"(9) avec un taux de couplage très faible, par ailleurs, le nombre de valeurs de ces deux index sont très proches (4503 pour nom_station contre 4459 pour coordonneesXY), ce qui signifie que dans la majorité des cas, on associe de façon unique une station et une position
- les exemples ci-dessous montrent les écarts les plus importants :
    - la position [1.106329, 49.474202] est associée à 10 stations
    - la station Camping Arinella est associée à 5 positions
    
- de même pour l'index "coordonneesXY"(13) qui est lié à l'index "adresse_station"(11), le taux de couplage est très faible avec 44 enregistrements sur 4456 en écart. On a par exemple quatre enregistrements avec la position (6.3491347, 47.3517596) et des adresses différentes ('58 Avenue du PrÃ©sident Kennedy 26', '58 Avenue du PrÃ©sident Kennedy 28', '58 Avenue du PrÃ©sident Kennedy 27', '58 Avenue du PrÃ©sident Kennedy 25')

In [11]:
champ = idxs.nindex
print('Couplage entre ', champ('coordonneesXY').name, ' et ', champ('nom_station').name, ' : ', 
      champ('coordonneesXY').couplinginfos(champ('nom_station'))['typecoupl'])
infosdefault = champ('nom_station').couplinginfos(champ('coordonneesXY'), default=True)
print('Ecart : ', infosdefault['disttomin'], 'positions sur ', infosdefault['distmin'], '\n') # moins de 1%

nom_station = champ('nom_station').tostdcodec(full=False)
coordonneesXY = champ('coordonneesXY').tostdcodec(full=False) 
coordonneesXY.coupling(nom_station)
c = Counter(coordonneesXY.codec).most_common(5)
print('les 5 positions avec le plus de stations: \n', c, '\n')
print('liste des stations associées à la position', c[0][0], ' :')
print(set([nom_station[i] for i in coordonneesXY.recordfromvalue(c[0][0])]), '\n')

coordonneesXY = champ('coordonneesXY').tostdcodec(full=False)
nom_station.coupling(coordonneesXY)
c = Counter(nom_station.codec).most_common(5)
print('les 5 stations avec le plus de positions: \n', c, '\n')
print('liste des positions associées à la station', c[0][0], ' :')
print(set([coordonneesXY[i] for i in nom_station.recordfromvalue(c[0][0])]))

Couplage entre  coordonneesXY  et  nom_station  :  linked
Ecart :  44 positions sur  4503 

les 5 positions avec le plus de stations: 
 [('[1.106329, 49.474202]', 10), ('[3.080477, 50.675889]', 6), ('[1.313367, 49.137233]', 6), ('[2.523685, 48.9908]', 5), ('[3.1557445, 50.5161745]', 4)] 

liste des stations associées à la position [1.106329, 49.474202]  :
{'SCH10', 'SCH08', 'SCH04', 'SCH03', 'SCH07', 'SCH01', 'SCH05', 'SCH02', 'SCH09', 'SCH06'} 

les 5 stations avec le plus de positions: 
 [('Camping Arinella', 5), ('GUERET', 4), ('Noues de Sienne, Le Bourg', 3), ('Chatellerault Nord', 2), ('900109', 2)] 

liste des positions associées à la station Camping Arinella  :
{'[9.445074, 41.995246]', '[9.445073, 41.995246]', '[9.445072, 41.995246]', '[9.445071, 41.995246]', '[9.445075, 41.995246]'}


In [12]:
print('Couplage entre ', champ('adresse_station').name, ' et ', champ('coordonneesXY').name, ' : ', 
      champ('adresse_station').couplinginfos(champ('coordonneesXY'))['typecoupl'])
infosdefault = champ('coordonneesXY').couplinginfos(champ('adresse_station'), default=True)
print('Ecart : ', infosdefault['disttomin'], 'positions sur ', infosdefault['distmin'], '\n') # moins de 1%

coordonneesXY = champ('coordonneesXY').tostdcodec(full=False)
adresse_station = champ('adresse_station').tostdcodec(full=False) 
adresse_station.coupling(coordonneesXY)
c = Counter(adresse_station.codec).most_common(5)
print('les 5 adresses avec le plus de positions : \n', c, '\n')
print('liste des position associées à l adresse :', c[0][0], ' :')
print(set([coordonneesXY[i] for i in adresse_station.recordfromvalue(c[0][0])]), '\n')

adresse_station = champ('adresse_station').tostdcodec(full=False)
coordonneesXY.coupling(adresse_station)
c = Counter(coordonneesXY.codec).most_common(5)
print('les 5 positions avec le plus d adresses : \n', c, '\n')
print('liste des adresses associées à la position', c[0][0], ' :')
print(set([adresse_station[i] for i in coordonneesXY.recordfromvalue(c[0][0])]))

Couplage entre  adresse_station  et  coordonneesXY  :  linked
Ecart :  41 positions sur  4459 

les 5 adresses avec le plus de positions : 
 [("rue de l'Ã©glise", 7), ('place de la mairie', 6), ('Place de la Mairie', 6), ('rue Grande', 5), ("place de l'Ã©glise", 5)] 

liste des position associées à l adresse : rue de l'Ã©glise  :
{'[3.2265338, 48.6845989]', '[2.76525, 48.448495]', '[2.9614005, 48.4189934]', '[3.3512623, 48.4528516]', '[2.5653149, 48.5803653]', '[3.343567, 48.693841]', '[2.7603113, 48.97101]'} 

les 5 positions avec le plus d adresses : 
 [('[6.3491347,47.3517596]', 4), ('[0.654826, 47.353958]', 4), ('[7.61,48.85]', 3), ('[0.9605582,49.5355585]', 3), ('[1.34918, 49.157966]', 3)] 

liste des adresses associées à la position [6.3491347,47.3517596]  :
{'58 Avenue du PrÃ©sident Kennedy 28', '58 Avenue du PrÃ©sident Kennedy 27', '58 Avenue du PrÃ©sident Kennedy 25', '58 Avenue du PrÃ©sident Kennedy 26'}


----
## Détection des incohérences de relations entre champs
- la fonction coupling permet de réorganiser la structure des relations en isolant les enregistrements incohérents
- dans le cas ci-dessous, elle est appliquée de façon automatique (minimisation des incohérences)
- les incohérences minimales sont dans le cas présent de 12%

In [13]:
idxs3 = Ilist.from_obj(cbor)
idxs3.coupling()
duplic = idxs3.getduplicates(resindex=ES.filter)
print('nombre d enregistrements incohérents : ', len(duplic), '\n')
idxs3.applyfilter()
print('nombre d enregistrements cohérents et liste des indexs non dérivés : ', len(idxs3), idxs3.primary)

nombre d enregistrements incohérents :  4757 

nombre d enregistrements cohérents et liste des indexs non dérivés :  6406 [15]


----
## Vérification simple
- on peut vérifier par exemple qu'une position est associée à une unique station et que réciproquement chaque station n'a qu'une seule position (relation 1-1 entre les deux champs).

In [14]:
idxs4 = copy(idxs2)
champ = idxs4.nindex
notcoupl = champ('coordonneesXY').coupling(champ('nom_station'), derived=False)
print('nombre de pdc avec position/station non couplées : ', len(notcoupl))
print('\nliste des premières incohérences : ')
liste = []
for i in range(100): 
    liste.append((champ('nom_station')[notcoupl[i]], champ('coordonneesXY')[notcoupl[i]]))
pprint(set(liste), width=120)

nombre de pdc avec position/station non couplées :  386

liste des premières incohérences : 
{(' IntermarchÃ© - Gauville - 22kW AC ', '[1.759704, 49.780879]'),
 (' Toyota - Montagnat - 22kW AC', '[5.2592028, 46.1746523]'),
 ('IntermarchÃ© - Gauville - 22kW AC ', '[1.759704, 49.780879]'),
 ('Lessafre1', '[3.080477, 50.675889]'),
 ('Lessafre2', '[3.080477, 50.675889]'),
 ('Lessafre3', '[3.080477, 50.675889]'),
 ('Lessafre4', '[3.080477, 50.675889]'),
 ('Lessafre5', '[3.080477, 50.675889]'),
 ('Lessafre6', '[3.080477, 50.675889]'),
 ('M2050', '[1.313367, 49.137233]'),
 ('M2051', '[1.313367, 49.137233]'),
 ('M2052', '[1.313367, 49.137233]'),
 ('M2060', '[1.313367, 49.137233]'),
 ('M2067', '[1.313367, 49.137233]'),
 ('M2070', '[1.313367, 49.137233]'),
 ('MECANIQUE SERVICES - Bonnelles', '[2.025665, 49.614061]'),
 ('PARKING MERMOZ 1', '[6.948066, 43.567243]'),
 ('Toyota - Annemasse - 22kW AC D ', '[6.2174783, 46.1858677]'),
 ('Toyota - Annemasse - 22kW AC G ', '[6.2174783, 46.1858677]'),
 ('

## Vérification par rapport à une structure imposée
- une autre utilisation possible est de vérifier les données par rapport à un modèle de données défini
- dans cet exemple, on peut regrouper les colonnes suivant quatre entités (ceci revient à considérer les colonnes comme des attributs de chacune des entités) : les opérateurs (colonne 6), les aménageurs (colonne 1), les stations (colonne 9), les pdc (colonne 15) -> cf quatre premières lignes ci-dessous
- on peut également indiquer les dépendances entre les quatre entités (les opérateurs et aménageurs sont dérivés par rapport aux stations qui elles sont dérivées par rapport aux pdc.

In [15]:
idxs.reindex()
idxs_impose = copy(idxs)
row_operateur   = [6,3,4,5]
row_amenageur   = [1,0,2]
row_station     = [9,10,11,12,13,14,23,24,25,26,27,28,29,30,31,32,33,34,35,36,43,44,45,46,47,48]
row_pdc         = [15,16,17,18,19,20,21,22,7,8,37,38,39,40,41,42]

champ = idxs_impose.lindex
operateur       = [champ[i] for i in row_operateur]
amenageur       = [champ[i] for i in row_amenageur]
station         = [champ[i] for i in row_station]
pdc             = [champ[i] for i in row_pdc]

pdc      [0].coupling(pdc      [1:])
station  [0].coupling(station  [1:])
amenageur[0].coupling(amenageur[1:])
operateur[0].coupling(operateur[1:])

station  [0].coupling([operateur[0], amenageur[0]])
pdc      [0].coupling(station[0])

print(idxs_impose.tree())

-1: root-derived (11163)
   15: id_pdc_itinerance (11154)
      7 : id_station_itinerance (7060)
      8 : id_station_local (3838)
      9 : nom_station (5185)
         1 : siren_amenageur (715)
            0 : nom_amenageur (481)
            2 : contact_amenageur (388)
         6 : nom_enseigne (860)
            3 : nom_operateur (152)
            4 : contact_operateur (159)
            5 : telephone_operateur (215)
         10: implantation_station (5)
         11: adresse_station (4392)
            28: condition_acces (3)
         12: code_insee_commune (2593)
         13: coordonneesXY (4459)
            33: station_deux_roues (7)
            43: consolidated_longitude (4448)
            44: consolidated_latitude (4429)
         14: nbre_pdc (32)
         23: gratuit (9)
         24: paiement_acte (9)
         25: paiement_cb (8)
         26: paiement_autre (9)
         27: tarification (128)
         29: reservation (8)
         30: horaires (140)
         31: accessibilite_pmr (5

La taille de ce format imposé (2,7 Mo) reste proche de la taille du format optimisé vu précédemment (2,4 Mo).

In [16]:
impose = idxs_impose.to_obj(encoded=True)
print('imposesize : ', len(impose), '\n')

imposesize :  2686687 



## Mise en cohérence des données
- l'application de la structure imposée permet d'identifier les enregistrements qui ne respectent pas la structure (cf exemples indiqués plus haut).
- dans l'exemple proposé, on identifie 48% des données ne respectant pas la structure
- on vérifie également qu'en supprimant ces données incoérentes, la structure définie est bien respectée

In [17]:
duplic = idxs_impose.getduplicates(idxs_impose.lname, ES.filter)
print('données respectant la structure (True) et ne la respectant pas (False) : ', 
      Counter(idxs_impose.lidx[49].values), '\n')
idxs_impose.applyfilter()
print(idxs_impose.tree())

données respectant la structure (True) et ne la respectant pas (False) :  Counter({True: 5775, False: 5388}) 

-1: root-derived (5775)
   15: id_pdc_itinerance (5775)
      7 : id_station_itinerance (3859)
      8 : id_station_local (1708)
      9 : nom_station (2424)
         6 : nom_enseigne (210)
            5 : telephone_operateur (37)
         11: adresse_station (2355)
            30: horaires (43)
         12: code_insee_commune (1470)
         13: coordonneesXY (2383)
            1 : siren_amenageur (357)
               0 : nom_amenageur (161)
                  28: condition_acces (2)
               2 : contact_amenageur (111)
            27: tarification (37)
            32: restriction_gabarit (35)
            39: last_modified (94)
               3 : nom_operateur (39)
               4 : contact_operateur (40)
               31: accessibilite_pmr (3)
               40: datagouv_dataset_id (82)
                  24: paiement_acte (6)
                  26: paiement_autre (6)
 